<a href="https://colab.research.google.com/github/DeekshaChowdhary/Covid_Fake_News_Classification/blob/main/Covid_Fake_News_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle
from sklearn.linear_model import LogisticRegressionCV
import re
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = "/content/drive/MyDrive/Colab Notebooks/covid_fake.csv"
df = pd.read_csv(file_path)

In [ ]:
df.head()

,title,text,source,label
0,Due to the recent outbreak for the Coronavirus...,"You just need to add water, and the drugs and ...",coronavirusmedicalkit.com,Fake
1,NaN,Hydroxychloroquine has been shown to have a 10...,RudyGiuliani,Fake
2,NaN,Fact: Hydroxychloroquine has been shown to hav...,CharlieKirk,Fake
3,NaN,The Corona virus is a man made virus created i...,JoanneWrightForCongress,Fake
4,NaN,Doesn’t @BillGates finance research at the Wuh...,JoanneWrightForCongress,Fake


In [ ]:
df.shape

(1164, 4)

In [ ]:
df['label'].value_counts()

,count
label,
TRUE,584
Fake,345
fake,230


In [ ]:
df.loc[5:15]

,title,text,source,label
5,CORONA UNMASKED: Chinese Intelligence Officer ...,NaN,NaN,NaN
6,NaN,Urgent: Health Bulletin to the Public. Ministr...,Ministry of Health,Fake
7,NaN,"Pls tell ur families, relatives and friendsMOH...",NWLLAB,Fake
8,NaN,SERIOUS EXCELLENT ADVICE by Japanese doctors t...,Japanese doctors treating COVID-19 cases,Fake
9,Basic protective measures against the new coro...,Stay aware of the latest information on the CO...,https://www.who.int/emergencies/diseases/novel...,TRUE
10,NaN,The new Coronavirus may not show signs of infe...,Taiwan Experts,Fake
11,NaN,A vaccine meant for cattle can be used to figh...,facebook,Fake
12,NaN,Using a hair dryer to breathe in hot air can c...,Youtube,Fake
13,NaN,Corona virus before it reaches the lungs it re...,twitter,Fake
14,Exposing yourself to the sun or to temperature...,"You can catch COVID-19, no matter how sunny or...",https://www.who.int/emergencies/diseases/novel...,TRUE


In [ ]:
df.isna().sum()

,0
title,82
text,10
source,20
label,5


In [ ]:
df.loc[df['label'] == 'Fake', 'label'] = 'FAKE'
df.loc[df['label'] == 'fake', 'label'] = 'FAKE'
df.loc[df['source'] == 'facebook', 'source'] = 'Facebook'
df['text'].fillna(df['title'], inplace=True)
df.loc[5, 'label'] = 'FAKE'
df.loc[15, 'label'] = 'TRUE'
df.loc[43, 'label'] = 'FAKE'
df.loc[131, 'label'] = 'TRUE'
df.loc[242, 'label'] = 'FAKE'
df = df.sample(frac=1).reset_index(drop=True)
df['title'].fillna('missing', inplace=True)
df['source'].fillna('missing', inplace=True)
df['title_text'] = df['title'] + " " + df['text']


In [ ]:
# Check for missing values again
df.isna().sum()

,0
title,0
text,0
source,0
label,0
title_text,0


In [ ]:
df.head()

,title,text,source,label,title_text
0,THE CORONAVIRUS AND THE HORIZON OF A MULTIPOLA...,The global coronavirus pandemic has huge geopo...,https://www.geopolitica.ru/,FAKE,THE CORONAVIRUS AND THE HORIZON OF A MULTIPOLA...
1,CORONA UNMASKED: Chinese Intelligence Officer ...,CORONA UNMASKED: Chinese Intelligence Officer ...,missing,FAKE,CORONA UNMASKED: Chinese Intelligence Officer ...
2,"TO MAINTAIN WORLD DOMINATION, A GROUP OF PEOPL...","In front of us, apparently, is the ideal crime...",missing,FAKE,"TO MAINTAIN WORLD DOMINATION, A GROUP OF PEOPL..."
3,"Yes, the coronavirus could have been completel...",Could the new coronavirus have been created in...,https://www.naturalnews.com/,FAKE,"Yes, the coronavirus could have been completel..."
4,No Evidence That Flu Shot Increases Risk of CO...,"First of all, experts say there has been no st...",https://www.factcheck.org/,TRUE,No Evidence That Flu Shot Increases Risk of CO...


In [ ]:
df['label'].value_counts()

,count
label,
TRUE,586
FAKE,578


In [ ]:
df.shape

(1164, 5)

In [ ]:
df['title_text'][3]

'Yes, the coronavirus could have been completely synthesized in a lab without leaving a trace Could the new coronavirus have been created in a lab? Writing in Medium, Youthereum Genetics CEO Yuri Deigin said that he believes it’s quite plausible. He writes that labs across the planet have been creating viruses just like this one for a long time, and while skeptics say that there would be some sign of human manipulation in its genome, modern genetic engineering tools enable genomic fragments to be cut and pasted without leaving a single trace. Not only that, but he insists it’s something that can be carried out quickly, citing the fact that a Swiss team created an authentic clone of CoV2 in under a month.At first, Deigin thought the idea of the virus being a lab creation was little more than a conspiracy theory. In fact, it was only when he was researching the situation to support his belief that he first started having doubts. One big red flag? As he looked further into the research ac

In [ ]:
def preprocessor(text):
  text = re.sub('<[^>]*>','',text)
  text = re.sub(r'[^w\s]','',text)
  text = re.sub(r'[\n]','',text)
  return text
df['title_text']= df['title_text'].apply(preprocessor)
df['title_text'][3]

'           w      w                        w                   w     w                      w                        w                        w          w  w  w                                  w         w                                       w             w                   w                               w                     w               w      w    w w   w         w         w                  w    w w          w           w     w         w              w           w  w      w     w      w  w     w       w                 w                                 w              w    w   w                                                    w    w     w w                                     w w              w                      w             w  w             '

In [ ]:
porter = PorterStemmer()
def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]

## TF-IDF


In [ ]:
# Replace 'preprocessing' with 'preprocessor'
tfid = TfidfVectorizer(strip_accents=None,
                      lowercase=False,
                      preprocessor=None,  # Changed 'preprocessing' to 'preprocessor'
                      tokenizer=tokenizer_porter,
                      use_idf=True,
                      norm='l2',
                      smooth_idf=True)
X = tfid.fit_transform(df['title_text'])
y = df.label.values

In [ ]:
X.shape

(1164, 5)

In [ ]:
X_train,X_test,y_train,y_test= train_test_split(X,y, random_state=0, \
                                                test_size=0.3, shuffle=False)

In [ ]:
clf = LogisticRegressionCV(cv=5, scoring="accuracy", random_state=0,n_jobs=-1, \
                           verbose=3, max_iter=300).fit(X_train,y_train)
clf.fit(X_train,y_train)
fake_news_model = open("fake_news_model.sav","wb")
pickle.dump(clf,fake_news_model)
fake_news_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    4.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.8s finished


In [ ]:
filename = 'fake_news_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))
saved_clf.score(X_test, y_test)

0.5314285714285715

In [ ]:
from sklearn.metrics import classification_report, accuracy_score
y_pred = clf.predict(X_test)
print("---Test Set Results----")
print(classification_report(y_test,y_pred))

---Test Set Results----
              precision    recall  f1-score   support

        FAKE       0.67      0.10      0.18       173
        TRUE       0.52      0.95      0.67       177

    accuracy                           0.53       350
   macro avg       0.59      0.53      0.43       350
weighted avg       0.59      0.53      0.43       350



In [ ]:
clf.predict(X_test[59])

array(['TRUE'], dtype=object)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import re

# ... (your preprocessor and tokenizer_porter functions) ...

# Define and fit tfidf in the current cell
tfid = TfidfVectorizer(strip_accents=None,
                      lowercase=False,
                      preprocessor=None,
                      tokenizer=tokenizer_porter,
                      use_idf=True,
                      norm='l2',
                      smooth_idf=True)
tfid.fit(df['title_text'])  # Assuming 'df' is also defined and in scope

# ... (your code for prediction) ...

TfidfVectorizer(lowercase=False,
                tokenizer=<function tokenizer_porter at 0x7d9a8d2ffce0>)

In [ ]:
test="Corona virus before it reaches the lungs"
inp = [test]
vect = tfidf.transform (inp)
prediction = clf.predict(vect)
print(prediction)

['FAKE']
